In [ ]:
import pandas as pd 
import sqlalchemy
from sqlalchemy import create_engine
import uuid 
import requests


# after creating a account in hasura/supabase, get the credentials and insert them here 
# assuming postgres DB 

# load data to insert 
data = pd.read_csv('./mountSinai_clean.csv')
data

,Unnamed: 0,hospital_name,insurance_type,code,code_description,cost_negotiated,cost_minimum,cost_maximum
0,0,MSH,AETNA AETNA CHOICE POS II NET 00357,1,Heart transplant or implant of heart assist sy...,56984.00,37063.00,898459.00
1,1,MSH,NYSDOH,1,Heart transplant or implant of heart assist sy...,37063.00,37063.00,898459.00
2,2,MSH,UNITEDHEALTHCARE OF NEW YORK INC NY DUAL COMPLETE,1,Heart transplant or implant of heart assist sy...,250698.00,37063.00,898459.00
3,3,MSH,AETNA ADV PLAN 250 NET 03397,1,Heart transplant or implant of heart assist sy...,222752.00,37063.00,898459.00
4,4,MSH,OXFORD HEALTH INSURANCE INC FREEDOM,1,Heart transplant or implant of heart assist sy...,798123.00,37063.00,898459.00
...,...,...,...,...,...,...,...,...
63392,63392,MSH,"WELLCARE OF NEW YORK, INC",U0004,SARSCOV2 CORONAVIRUS HIGH,103.00,38.50,389.00
63393,63393,MSH,"WellMed Medical Management, Inc.",U0004,SARSCOV2 CORONAVIRUS HIGH,100.00,38.50,389.00
63394,63394,MSH,WMMI,U0004,SARSCOV2 CORONAVIRUS HIGH,100.00,38.50,389.00
63395,63395,MSH,EH FACET EFT,V2632,LENS ACRYSOF IQ IOL 23.0D,178.72,178.72,202.95


In [ ]:
!pip install psycopg2-binary==2.8.6

     |████████████████████████████████| 3.0 MB 5.5 MB/s 


In [ ]:

####INITIAL FOR INSERTION OF UIDs
data['uuid'] = [uuid.uuid4() for _ in range(len(data.index))]
data


,Unnamed: 0,hospital_name,insurance_type,code,code_description,cost_negotiated,cost_minimum,cost_maximum,uuid
0,0,MSH,AETNA AETNA CHOICE POS II NET 00357,1,Heart transplant or implant of heart assist sy...,56984.00,37063.00,898459.00,a670c3d4-4166-412e-a5de-cf63687de9de
1,1,MSH,NYSDOH,1,Heart transplant or implant of heart assist sy...,37063.00,37063.00,898459.00,8ad65b5d-9adf-43ab-9bf5-fa1508a8db25
2,2,MSH,UNITEDHEALTHCARE OF NEW YORK INC NY DUAL COMPLETE,1,Heart transplant or implant of heart assist sy...,250698.00,37063.00,898459.00,b6b42741-ceab-4d9b-8370-e89990f86eb4
3,3,MSH,AETNA ADV PLAN 250 NET 03397,1,Heart transplant or implant of heart assist sy...,222752.00,37063.00,898459.00,9ec8d22d-598a-4a8c-9614-d0b43b39aa23
4,4,MSH,OXFORD HEALTH INSURANCE INC FREEDOM,1,Heart transplant or implant of heart assist sy...,798123.00,37063.00,898459.00,a46bd5a6-550a-47a4-a41f-7bce129964cb
...,...,...,...,...,...,...,...,...,...
63392,63392,MSH,"WELLCARE OF NEW YORK, INC",U0004,SARSCOV2 CORONAVIRUS HIGH,103.00,38.50,389.00,63bc6be8-9986-4b0f-be42-bdb24b3d1c1b
63393,63393,MSH,"WellMed Medical Management, Inc.",U0004,SARSCOV2 CORONAVIRUS HIGH,100.00,38.50,389.00,8e5313cb-b233-4ae2-80db-473eda7d5347
63394,63394,MSH,WMMI,U0004,SARSCOV2 CORONAVIRUS HIGH,100.00,38.50,389.00,44cd8aa3-4ae2-444f-9179-9b76a9860f2c
63395,63395,MSH,EH FACET EFT,V2632,LENS ACRYSOF IQ IOL 23.0D,178.72,178.72,202.95,af5dd14e-9327-49f7-8110-13c32dc3136d


In [ ]:

####HASURA or SUPABASE 

sqlUrl = sqlalchemy.engine.url.URL(
    drivername="postgresql+psycopg2",
    username='postgres',
    password='hospitalpricing2022',
    host='db.xfeadtafhymsoresjzjn.supabase.co',
    #port=6543,
    port=5432,
    database='postgres',
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
engine = create_engine(sqlUrl)



In [ ]:
# test engine
engine.connect()



In [ ]:

# # just keep first 5 rows for testing
# data = data.head(5)
# data


,Unnamed: 0,hospital_name,insurance_type,code,code_description,cost_negotiated,cost_minimum,cost_maximum,uuid
0,0,MSH,AETNA AETNA CHOICE POS II NET 00357,1,Heart transplant or implant of heart assist sy...,56984.0,37063.0,898459.0,8cd5ae74-1a9c-4c73-9ee6-2525bd01947a
1,1,MSH,NYSDOH,1,Heart transplant or implant of heart assist sy...,37063.0,37063.0,898459.0,875f4583-838e-4820-8edb-cdd13e7d5874
2,2,MSH,UNITEDHEALTHCARE OF NEW YORK INC NY DUAL COMPLETE,1,Heart transplant or implant of heart assist sy...,250698.0,37063.0,898459.0,06b0196b-69c5-4436-8872-0aac2182f6b6
3,3,MSH,AETNA ADV PLAN 250 NET 03397,1,Heart transplant or implant of heart assist sy...,222752.0,37063.0,898459.0,caa2652e-c6a2-48d8-8af7-95a54625599a
4,4,MSH,OXFORD HEALTH INSURANCE INC FREEDOM,1,Heart transplant or implant of heart assist sy...,798123.0,37063.0,898459.0,dcb0b14c-0998-4143-bf70-834d7cbe7e28


In [ ]:
####INSERT DATA INTO HASURA/SUPABASE
data.to_sql('mountsinai', con=engine, if_exists='replace')